In [4]:
import scipy.stats as stats
import numpy as np
import pandas as pd
from tkinter import filedialog
import matplotlib.pyplot as plt
import seaborn as sns
import os
from math import ceil
from rich.progress import track

# Disable chained assignment warning
pd.options.mode.chained_assignment = None

In [5]:
um_per_pixel = 0.117
s_per_frame = 2
window_size = 10
dtype_dict = {
    "POSITION_T": "float64",
    "POSITION_X": "float64",
    "POSITION_Y": "float64",
    "TRACK_ID": "str",
}

In [6]:
# File selection dialog (make sure to run this in an environment that supports dialogs, or replace with a direct file path)
csv_file_path = filedialog.askopenfilename(
    title="Select CSV File",
    filetypes=(("CSV files", "*.csv"), ("All files", "*.*")),
)


In [7]:
# Read the CSV file with predefined data types
df = pd.read_csv(csv_file_path, dtype=dtype_dict)

# Data cleaning and type conversion
df = df.dropna(subset=["POSITION_T", "POSITION_X", "POSITION_Y"])
df["TRACK_ID"] = pd.to_numeric(df["TRACK_ID"], errors="coerce").astype("Int64")
df = df.dropna(subset=["TRACK_ID"])

# Data scaling
df["POSITION_T"] *= s_per_frame
df["POSITION_X"] *= um_per_pixel
df["POSITION_Y"] *= um_per_pixel

df[['TRACK_ID', 'POSITION_X', 'POSITION_Y', 'POSITION_T']]

,TRACK_ID,POSITION_X,POSITION_Y,POSITION_T
3,0,11.529929,71.451753,40.0
4,0,11.502791,71.461115,38.0
5,0,11.552062,71.451755,48.0
6,0,11.680944,71.437151,92.0
7,0,11.794341,71.473277,120.0
...,...,...,...,...
257183,5101,14.442814,46.580338,396.0
257184,5101,14.511472,46.582978,394.0
257185,5101,14.466496,46.588714,392.0
257186,5101,14.474264,46.556897,390.0


In [8]:
#groupby and sort operation:

grouped_tracks = df.groupby('TRACK_ID').apply(lambda x: x.sort_values('POSITION_T'))

for track_id, track_data in grouped_tracks.groupby(level=0):
    print (f"TRACK_ID: {track_id}")
    print (track_data.head())
    break

TRACK_ID: 0
                LABEL        ID  TRACK_ID     QUALITY  POSITION_X  POSITION_Y  \
TRACK_ID                                                                        
0        58  ID194024  194024.0         0  445.762909   11.375501   71.418895   
         44  ID188502  188502.0         0  515.661560   11.388848   71.422333   
         11  ID194826  194826.0         0  508.869904   11.380167   71.409267   
         17  ID205456  205456.0         0  540.337585   11.356968   71.415346   
         9   ID204165  204165.0         0  575.240234   11.398466   71.431427   

             POSITION_Z  POSITION_T  FRAME  RADIUS  VISIBILITY  \
TRACK_ID                                                         
0        58         0.0         0.0    0.0     1.5         1.0   
         44         0.0         2.0    1.0     1.5         1.0   
         11         0.0         4.0    2.0     1.5         1.0   
         17         0.0         6.0    3.0     1.5         1.0   
         9          0.0 

In [9]:
def calc_MSD_NonPhysUnit(track_data, lags):

    Xs = track_data["POSITION_X"].to_numpy()
    Ys = track_data["POSITION_Y"].to_numpy()

    MSDs = []
    for lag in lags:
        displacements = (Xs[:-lag] - Xs[lag:]) ** 2 + (Ys[:-lag] - Ys[lag:]) ** 2
        valid_displacements = displacements[~np.isnan(displacements)]
        MSD = np.nanmean(valid_displacements)
        MSDs.append(MSD)

    return np.array(MSDs, dtype=float)

In [10]:
for track_id, track_data in grouped_tracks.groupby(level=0):

    lags = np.arange(1, (window_size + 1))
    MSDs = calc_MSD_NonPhysUnit(track_data, lags)

    print(f"TRACK_ID : {track_id}")
    print(f"MSD: {MSDs}\n")

    break

TRACK_ID : 0
MSD: [0.0009017  0.00134358 0.00170691 0.00217601 0.00253843 0.00328366
 0.00419797 0.00516414 0.00607274 0.00710316]



In [11]:
def calc_alpha (MSDs, lags):

    valid_indices = ~np.isnan(MSDs)
    valid_MSDs = MSDs [valid_indices]
    valid_lags = lags [valid_indices]

    log_lags = np.log10 (valid_lags)
    log_MSDs = np.log10 (valid_MSDs)

    slope, intercept, r_value, p_value, std_error = stats.linregress(log_lags, log_MSDs)

    alpha = slope
    diffusion_coefficient = (1/4) * (10 ** intercept)

    return alpha, r_value ** 2, diffusion_coefficient



In [12]:
for track_id, track_data in grouped_tracks.groupby(level=0):
    lags = np.arange (1, (window_size+1))
    MSDs = calc_MSD_NonPhysUnit (track_data, lags)

    if not np.any(np.isnan(MSDs)):
        alpha, r_squared, diffusion_coefficient = calc_alpha (MSDs, lags)

        print(f"TRACK_ID: {track_id}")
        print(f"Alpha: {alpha}")
        print(f"R^2: {r_squared}")
        print(f"Diffusion Coefficient: {diffusion_coefficient}\n")

    break


TRACK_ID: 0
Alpha: 0.9048160206104549
R^2: 0.9497884545290923
Diffusion Coefficient: 0.00018151038552422007



In [13]:
def calculate_alpha_for_track(df_track, um_per_pixel, s_per_frame, window_size):

    df_track["R2"] = np.nan
    df_track["alpha"] = np.nan
    df_track["D"] = np.nan

    window_info = []

    # Iterate through windows and update alpha for the middle frame
    step_size = 1
    for start in range(0, len(df_track) - window_size + 1, step_size):
        end = start + window_size
        df_window = df_track.iloc[start:end]

        number_lag = ceil(window_size / 2)
        if number_lag < 3:
            number_lag = 3
        window_msd = calc_MSD_NonPhysUnit(
            df_track.iloc[start:end], np.arange(1, number_lag + 1)
        )
        if np.sum(window_msd <= 0) > 0:
            # Skip this window since it contains invalid MSD values
            continue

        alpha, r_value, D = calc_alpha(
            window_msd, np.arange(1, number_lag + 1) * s_per_frame
        )
        r_squared = r_value**2
        if not np.isnan(alpha) and r_squared > 0.9:
            middle_frame_index = start + ceil(window_size / 2)
            df_track.at[df_track.index[middle_frame_index], "R2"] = r_squared
            df_track.at[df_track.index[middle_frame_index], "alpha"] = alpha
            df_track.at[df_track.index[middle_frame_index], "D"] = D

            window_info.append(
                {
                    "window_start_frame": df_window.iloc[0]["POSITION_T"],
                    "window_end_frame": df_window.iloc[-1]["POSITION_T"],
                    "alpha": alpha,
                    "R2": r_squared,
                    "D": D,
                    "lags": np.arange(1, number_lag + 1).tolist(),
                }
            )

    return df_track, window_info

In [14]:
for track_id, df_track in grouped_tracks.groupby(level=0):

    result_df_track, window_info = calculate_alpha_for_track(
        df_track, um_per_pixel, s_per_frame, window_size
    )
    calculated = result_df_track.dropna(subset=["alpha", "R2", "D"])

    print(f"Track_id: {track_id}")

    if not calculated.empty:
        print(
            calculated[["POSITION_X", "POSITION_Y", "POSITION_T", "R2", "alpha", "D"]]
        )
        for info in window_info:
            print(
                f"Window from {info['window_start_frame']} to {info['window_end_frame']}s: lags = {info['lags']}"
            )
            print(f"Alpha: {info['alpha']}, R2: {info['R2']}, D: {info['D']}\n")

    print("\n")

    break

Track_id: 0
             POSITION_X  POSITION_Y  POSITION_T        R2     alpha         D
TRACK_ID                                                                     
0        23   11.408236   71.434286        12.0  0.919300  0.839650  0.000083
         54   11.400308   71.433952        14.0  0.956228  0.929528  0.000081
         61   11.424691   71.451955        16.0  0.913374  1.084230  0.000066
         33   11.440764   71.425429        18.0  0.947688  1.129834  0.000054
         45   11.459172   71.412223        20.0  0.966906  1.115575  0.000058
         57   11.458940   71.441887        22.0  0.958226  1.046821  0.000067
         60   11.470620   71.430901        24.0  0.975375  1.018079  0.000076
         32   11.470015   71.463525        26.0  0.941003  0.999790  0.000064
         13   11.490374   71.491709        32.0  0.980718  0.983215  0.000058
         21   11.484714   71.481789        34.0  0.968838  1.067802  0.000055
         63   11.494441   71.484536        36.0  0.9

In [15]:
def process_csv_and_add_alpha_and_D(
    csv_file_path, window_size, um_per_pixel, s_per_frame
):

    save_path = os.path.dirname(csv_file_path)
    base_name = os.path.basename(csv_file_path)
    name, ext = os.path.splitext(base_name)
    output_file_name = f"{name}_alpha_and_D_w{window_size}{ext}"
    output_file_path = os.path.join(save_path, output_file_name)

    df = pd.read_csv(csv_file_path, dtype=dtype_dict)
    df["POSITION_T"] *= s_per_frame

    for column in ["R2", "alpha", "D"]:
        if column not in df:
            df[column] = np.nan

    grouped_tracks = df.groupby("TRACK_ID")
    for track_id, track_df in grouped_tracks:
        processed_track, window_info = calculate_alpha_for_track(
            track_df, um_per_pixel, s_per_frame, window_size
        )
        df.loc[processed_track.index] = processed_track

    df.sort_values(by=["TRACK_ID", "POSITION_T"], inplace=True)

    df.to_csv(output_file_path, index=False)
    print(f"Processed file saved : {output_file_path}")

    return output_file_path


process_csv_and_add_alpha_and_D(csv_file_path, window_size, um_per_pixel, s_per_frame)

Processed file saved : Z:/Bisal_Halder_turbo/PROCESSED_DATA/Trial_analysis/Noco/2x\20240116_UGD-2x-2s-replicate1-FOV_alpha_and_D_w10.csv


'Z:/Bisal_Halder_turbo/PROCESSED_DATA/Trial_analysis/Noco/2x\\20240116_UGD-2x-2s-replicate1-FOV_alpha_and_D_w10.csv'